In [ ]:


from PIL import Image,ImageOps,ImageStat,ImageDraw, ImageFilter
import os,os.path
import sys
import math
import tensorflow as tf
from tensorflow  import keras as keras
from keras import Model
from keras import layers
from keras import losses
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
#from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import random
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

#psp_fileName = "D:/image_data/__FR1.jpg.BMP._PSP"
#image_fileName = "D:/image_data/__FR1.jpg.BMP"
#newMask_fileName = "D:/image_data/new_mask_class_"
#fragm_dir = "D:/image_data/fragments"
#acc_file = 'D:/image_data/accuracy.txt'
#result_fileName = "D:/image_data/classification_result.BMP"
#marks_filename = 'D:/image_data/class_marks.txt'
#img_len = 512
#param1 = 25

img_file_name = image_fileName
#дополнять массив цветами по мере необходимости\n",
t_color = ['blue', 'red', 'yellow', 'green' , 'violet']
temple_file = Image.open(img_file_name)
colors = {}
#сборка границ прямоугольникв, из которых будут вырезаться области (если их на изображении несколько)
classRects=[]
curColor = ""
class_numb = '021'
rect_flag  = False
load_in_list = False
gen_crop_img = False
class_amount = 5
step_len = 5
win_size = step_len * 2 + 1
bound = math.ceil(math.sqrt(2) * win_size - win_size)
dir_name = fragm_dir

img = Image.new('RGB', temple_file.size, color=0)
draw = ImageDraw.Draw(img)
fp = open(psp_fileName)

for k, txt in enumerate(fp):
    if k<2:
        continue
    zz, t = txt.split('=')
    if t.find('Pline') >= 0:
        x2y = []
        rect_flag = False
        continue
    if t.find('Rectangle') >= 0:
        x2y = []
        rect_flag = True
        continue
    if t.find('Pen') >= 0:
        p = t.split(',')
        curColor = p[2]
        if p[2] not in colors:
            colors.update({p[2]: len(colors)})
    t = t.split(' ')
    if t[0] == '':
        if rect_flag:
            x2y.insert(1, (x2y[0][0], x2y[1][1]))
            x2y.append((x2y[2][0], x2y[0][1]))
        idx = colors.get(curColor) 
        if idx ==  len(classRects):
            classRects.append([x2y])
        else:
            classRects[idx].append(x2y)
        print(classRects)   
        draw.polygon(x2y, outline = t_color[idx], fill = t_color[idx])
        continue
    x2y.append((int(t[0]), int(t[1])))

img.save(newMask_fileName + class_numb +".BMP")
img.close()
fp.close()







In [ ]:
labels=[]
nmb_fragm = 0
print(bound)
im = Image.open(img_file_name)
for i in range(class_amount):
    for j in range(len(classRects[i])):
        x0 = classRects[i][j][0][0]
        y0 = classRects[i][j][0][1]
        x_len = classRects[i][j][1][1]-classRects[i][j][0][1]
        y_len = classRects[i][j][2][0]-classRects[i][j][1][0]
        x_amount = int(x_len/step_len)
        y_amount = int(y_len/step_len)
        for k in range(x_amount):
            for l in range(y_amount):
                labels.append(i)
                im2 = im.crop((x0 + k * step_len - bound, y0+ l * step_len - bound, x0 + k * step_len +bound + win_size, y0+ l * step_len +bound + win_size))
                imfnam = dir_name+"/+++fragm{:05d}.jpg".format(nmb_fragm);
                nmb_fragm += 1;
                im2.save(imfnam)
im.close()

In [ ]:
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

temple_file = Image.open(img_file_name)
width, height = temple_file.size
width_re, height_re = temple_file.size
width_re = width_re - width_re % 16
height_re = height_re - height_re % 16
temple_file.close()

if flag_gen == True:
    def img_acuumulation_func(resourse_filename, result_filename):
        picture = Image.open(resourse_filename)
        width_p, height_p = picture.size
        width_p = width_p // 2
        height_p = height_p // 2
        img_accumulated = Image.new('RGB', (width_p, height_p), color=0)
        im2 = np.zeros((1, 2, 2, 3), dtype='float32')
        for i in range(width_p):
            for j in range(height_p):
                if (i + 2) < width_p and (j + 2) < height_p:
                    im2[0] = np.array(picture.crop((2 * i, 2 * j, 2 * i + 2, j * 2 + 2)))
                    color = (im2[0][0, 0] + im2[0][0, 1] + im2[0][1, 1] + im2[0][1, 0])//4
                    img_accumulated.putpixel((i, j), (color[0], color[1], color[2]))
        img_accumulated.save(result_filename)
        img_accumulated.close()
        picture.close()


    img_acuumulation_func(img_file_name, glob_dir_name + "accumulated.BMP")
    img_acuumulation_func(glob_dir_name + "accumulated.BMP", glob_dir_name + "accumulated_0.BMP")
    img_acuumulation_func(glob_dir_name + "accumulated_0.BMP", glob_dir_name + "accumulated_1.BMP")


In [ ]:
gen_crop_img = True

if gen_crop_img:
    def croppingFunction(resource_name, width, height, list_file, fragm_dir):
        img = Image.open(resource_name)
        nmb_fragm = 0
        print(width, height)
        with open (list_file, 'wt') as file:
            for i in range(0, width, step_len - shift_draw):
                for j in range(0, height, step_len - shift_draw):
                    if(i + win_size) < width and (j + win_size) < height:
                        #x2y = [(i, j), (i + win_size, j), (i + win_size, j + win_size), (i, j + win_size)]
                        im2 = img.crop((i, j, i + win_size, j + win_size))
                        imfnam = fragm_dir + "+++fragm{:05d}.jpg".format(nmb_fragm)
                        file.write(imfnam + " " + str(i) + " " + str(j) + "\n")
                        nmb_fragm += 1;
                        im2.save(imfnam)
                    else:
                        i_1, j_1 = i, j
                        if i + win_size >= width:
                            i_1 = width - win_size
                        if j + win_size >= height:
                            j_1 = height - win_size
                        im2 = img.crop((i_1, j_1, i_1 + win_size, j_1 + win_size))
                        imfnam = fragm_dir + "+++fragm{:05d}.jpg".format(nmb_fragm)
                        file.write(imfnam + " " + str(i_1) + " " + str(j_1) + "\n")
                        nmb_fragm += 1;
                        im2.save(imfnam)
        img.close()

    #croppingFunction(img_file_name, width_re, height_re, "D:/_SAR_Kubinka/list_of_fragments_0.txt",
                 #"D:/_SAR_Kubinka/cropped_img/_0")
    '''
    croppingFunction(glob_dir_name + "accumulated.BMP", width_re // 2, height_re // 2,
                     "D:/_SAR_Kubinka/list_of_fragments_1.txt",
                     "D:/_SAR_Kubinka/cropped_img/_1")
    croppingFunction(glob_dir_name + "accumulated_0.BMP", width_re // 4, height_re // 4,
                     "D:/_SAR_Kubinka/list_of_fragments_2.txt",
                     "D:/_SAR_Kubinka/cropped_img/_2")
    croppingFunction(glob_dir_name + "accumulated_1.BMP", width_re // 8, height_re // 8,
                     "D:/_SAR_Kubinka/list_of_fragments_2.txt",
                     "D:/_SAR_Kubinka/cropped_img/_3")
                     
                     '''
    croppingFunction(glob_dir_name + "accumulated_1.BMP", width_re // 8, height_re // 8,
                     "D:/_SAR_Kubinka/list_of_fragments_3.txt",
                     "D:/_SAR_Kubinka/cropped_img/_3")
# In[91]:


In [ ]:

load_in_list = True
if load_in_list:
    def load_in_list_idx(list_file):
        img_list = []
        img_coords=[]
        with open(list_file, 'r') as file:
            for bf in file:
                bf_list = bf.split(" ")
                img_list.append(bf_list[0])
                img_coords.append((int(bf_list[1]), int(bf_list[2])))
        file.close()
        return img_list, img_coords


    img_list_0, img_coords_0 = load_in_list_idx("D:/_SAR_Kubinka/list_of_fragments_0.txt")
    img_list_1, img_coords_1 = load_in_list_idx("D:/_SAR_Kubinka/list_of_fragments_1.txt")
    img_list_2, img_coords_2 = load_in_list_idx("D:/_SAR_Kubinka/list_of_fragments_2.txt")
    img_list_3, img_coords_3 = load_in_list_idx("D:/_SAR_Kubinka/list_of_fragments_3.txt")
# In[102]:


In [ ]:



#rjkbxtcndj преобразований , выполняемых над изображением
transAmount = 6;
dlin   = 0

def load_CNN_train_augment(dir_name1):
    ll = []           #пустой список имен входных файлов JPG
    for file in os.listdir(dir_name1):
        if file.endswith(".jpg"): ll.append(file)
    dlin   = len(ll)     
    dlin0  = int(len(ll)/2)  #будем пропускать нечетные элементы
    train_x = np.zeros((dlin * transAmount, win_size, win_size, 3), dtype='float32')
    print('train_x_augm.shape=',train_x.shape)
    train_z = np.zeros((dlin * transAmount, 1), dtype='float32')
    #dlin = dlin * transAmount
    k=0
    for file in ll:
        im = Image.open(dir_name+"\\"+file)
        train_x[transAmount * k] = np.array(im.crop((bound, bound, bound + win_size, bound + win_size)))*1./255.
        train_x[transAmount * k + 1] = np.array(im.rotate(45).crop((bound, bound, bound + win_size,bound + win_size)))*1./255.    #print(b0.shape)
        shift0 = random.randint(0,6)
        shift1 = random.randint(0,6)
        shift2 = random.randint(0,6)
        shift3 = random.randint(0,6)
        if shift0 == 0 and shift1 == 0:
            shift0 += 2
        if shift2 == 0 or shift2 == 3:    
            shift2 +=1
        if shift3 == 0 or shift3 == 3:    
            shift3 +=1    
        train_x[transAmount * k + 2] = np.array(im.crop((shift0, 0, shift0 + win_size, win_size)))*1./255.    #print(b0.shape)
        
        train_x[transAmount * k + 3] = np.array(im.crop((0, shift1, win_size, shift1 + win_size)))*1./255.    #print(b0.shape)
        train_x[transAmount * k + 4] = np.array(im.rotate(90).crop((shift2, shift3, shift2 + win_size, shift3 + win_size))) * 1./255.
        train_x[transAmount * k + 5] = np.array(im.rotate(180).crop((bound, bound, bound + win_size, bound+win_size)))*1./255.
        #train_x[transAmount * k + 4] = np.array(im.crop((0,8,13,21)))*1./255.
        for p in range(transAmount):
            #print(k,p)
            train_z[k * transAmount + p] = int(labels[k])

        k+=1
        if k == dlin: break
    temp=[]
    for i in range(len(train_z)):
        temp.append(to_categorical(train_z[i], num_classes=class_amount))
    train_y=np.array(temp)    
    return train_x,train_y

train_x1,train_y1=load_CNN_train_augment(dir_name)
get_ipython().run_line_magic('matplotlib', 'inline')

def show_img_data(X_data,K0,rows,cols):   # смотрим на первые rows*cols 
    fig, axes = plt.subplots(rows, cols, figsize=(4, 2), dpi=160) #, figsize=(20, 6)) 
    for j in range(rows):
        for k in range(cols):
            axes[j,k].set_axis_off()
            axes[j,k].imshow(X_data[K0+j*cols+k].squeeze(), cmap='Greys', interpolation='None') #)'auto')

show_img_data(train_x1,0,4,8) 


In [ ]:
amount_valid = 1000
k = 0
idx_del = set()
train_x_valid = np.zeros((amount_valid, win_size, win_size, 3), dtype='float32')
train_y_valid = np.zeros((amount_valid,1,class_amount), dtype='float32')
while k <amount_valid:
    idx =random.randint(0, train_x1.shape[0]-1)
    while idx in idx_del:
        idx = random.randint(0, train_x1.shape[0]-1)
    idx_del.add(idx)
    train_x_valid[k] = train_x1[idx]
    train_y_valid[k] = train_y1[idx]
    k += 1



idx_del = list(idx_del)
train_x_data = np.delete(train_x1, idx_del, 0)
train_y_data = np.delete(train_y1, idx_del, 0)

print('train_x_augm.shape=', train_y_valid.shape)
print('train_x_augm.shape=', train_x_data.shape)
#реорганизация набора тестовых данных, чтобы была равномерность 
train_x_valid_buf = np.zeros((train_x_data.shape[0] // 2, win_size, win_size, 3), dtype='float32')
train_y_valid_buf = np.zeros((train_x_data.shape[0] // 2, 1, class_amount), dtype='float32')

for i in range(train_x_data.shape[0] // 2):
    if i % 2 == 0:
        train_x_valid_buf[i] = train_x_data[2 * i]
        train_y_valid_buf[i] = train_y_data[2 * i]
np.flip(train_x_valid_buf)
np.flip(train_y_valid_buf)
for i in range(train_x_data.shape[0] // 2):
    if i % 2 == 0:
        train_x_data[2 * i] = train_x_valid_buf[i]
        train_y_data[2 * i] = train_y_valid_buf[i]

In [ ]:

def train_CNN_model_ex(model_t, train_xt, train_yt, dlin_t, msize, step1, step2, val_split):
    model_temp = model_t
    val_acc_min = 0.0
    for k in range(step1):
        #model_t.fit(train_xt,train_yt,epochs=step2,verbose=0,batch_size=16,validation_split=0.15)
        hist = model_t.fit(train_xt, train_yt.squeeze(), epochs=step2, verbose=0, validation_split=val_split)
        vvv = hist.history['val_accuracy']
        print(k, vvv)
        val_acc = vvv[0]
        if val_acc > val_acc_min:
            model_temp = model_t
            val_acc_min = val_acc
            hist = model_t.fit(train_xt, train_yt.squeeze(), epochs=1, verbose=0, validation_split=val_split)

    print('end CNN train')
    return model_temp


# In[106]:



num_train, height_data, width_data, depth_data = train_x_data.shape # there are 50000 training examples in CIFAR-10
print(num_train, height_data, width_data, depth_data)



In [ ]:


def def_cnn_model_0 (l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 6#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_1(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 22#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_2(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes=class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_3(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 12#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes=class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_4(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 12#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes=class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_1)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_5(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 22#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_1)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_6(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 22#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='sigmoid')(conv_1)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_7(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 6#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes=class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_8(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 6#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes=class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='selu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_9(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 6#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='sigmoid')(conv_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


# In[ ]:


def def_cnn_model_10(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 6#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='sigmoid')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model

def def_cnn_model_11(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 15#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation="relu")(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    conv_3 = Convolution2D(5, kernel_size, kernel_size, border_mode='same', activation="relu")(drop1)
    conv_4 = Convolution2D(5, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


def def_cnn_model_12(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 15#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(5, kernel_size, kernel_size, border_mode='same', activation="relu")(inp)
    conv_2 = Convolution2D(5, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    conv_3 = Convolution2D(15, kernel_size, kernel_size, border_mode='same', activation="relu")(drop1)
    conv_4 = Convolution2D(15, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


def def_cnn_model_13(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 15#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(5, kernel_size, kernel_size, border_mode='same')(inp)
    conv_2 = Convolution2D(5, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    conv_3 = Convolution2D(15, kernel_size, kernel_size, border_mode='same')(drop1)
    conv_4 = Convolution2D(15, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    out = Dense(num_classes, activation='softmax')(flat)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model


def def_cnn_model_14(l_height, l_width, l_depth):
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 15#6#height*2 # we will initially have 32 kernels per conv. layer...
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    num_classes = class_amount

    inp = Input(shape=(l_height, l_width, l_depth)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(5, kernel_size, kernel_size, border_mode='same')(inp)
    conv_2 = Convolution2D(5, kernel_size, kernel_size, border_mode='same')(conv_1)
    
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    conv_3 = Convolution2D(15, kernel_size, kernel_size, border_mode='same')(drop1)
    conv_4 = Convolution2D(15, kernel_size, kernel_size, border_mode='same')(conv_3)
    
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    out1 = Dense(15, activation='relu')(flat)
    out2 = Dense(15, activation='relu')(out1)
    out = Dense(num_classes, activation='softmax')(out2)

    l_model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    l_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return l_model
# In[ ]:



In [ ]:

model_creation_funcs = [def_cnn_model_0, def_cnn_model_1,def_cnn_model_2,def_cnn_model_3,
                        def_cnn_model_4,def_cnn_model_5,def_cnn_model_6, def_cnn_model_7,
                        def_cnn_model_8, def_cnn_model_9, def_cnn_model_10, def_cnn_model_11,
                        def_cnn_model_12, def_cnn_model_13, def_cnn_model_14]

msize=16 
model=model_creation_funcs[idx_model](height_data, width_data, depth_data)

model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,1,0.1)

In [ ]:


def classification_sequential(model):
    image_full = Image.open(img_file_name)
    img_new = Image.new('RGB', image_full.size, color=0)
    x0 = 0
    y0 = 0
    draw = ImageDraw.Draw(img_new)
    width, height = img_new.size
    im2 = np.zeros((1, win_size, win_size, 3), dtype='float32')
    class_marks = np.zeros((width, height), dtype='int')
    for i in range(0, width, step_len - shift_draw):
        for j in range(0, height, step_len - shift_draw):
            if(i + win_size) < width and (j + win_size ) < height:
                x2y = [(i, j), (i + win_size, j), (i + win_size, j + win_size), (i, j + win_size)]
                im2[0] = np.array(image_full.crop((i, j, i + win_size, j + win_size)))*1./255.
                z = model.predict(im2, verbose=0)
                m_idx = np.argmax(z[0])
                for k in range(win_size):
                    for l in range(win_size):
                        class_marks[i + k, j + l] = m_idx
                draw.polygon(x2y, outline=t_color[m_idx], fill=t_color[m_idx])

    img_new.save(result_fileName + str(idx_model) + ".BMP")
    img_new.close()


def classification_by_batch(model, img_list, img_coords):
    temple_file = Image.open(img_file_name)
    img_new = Image.new('RGB', temple_file.size, color=0)
    draw = ImageDraw.Draw(img_new)
    width, height = img_new.size
    class_marks = np.zeros((width, height), dtype='int')
    batch_size_t = 64
    batch_count = 0
    batch_pack = np.zeros((batch_size_t, win_size, win_size, 3), dtype='float32')
    batch_coords = []
    idx_counter = 0

    for i in img_list:

        if (batch_count < batch_size_t):
            im = Image.open(i)
            batch_pack[batch_count] = np.array(im) * 1. / 255.
            batch_coords.append(img_coords[idx_counter])
            batch_count += 1
            idx_counter += 1
            im.close()
        else:
            pred_res = model.predict(batch_pack, batch_size=batch_size_t, verbose=0)
            for j in range(pred_res.shape[0]):
                i_idx = batch_coords[j][0]
                j_idx = batch_coords[j][1]
                max_idx = np.argmax(pred_res[j])


                for k in range(win_size):
                    for l in range(win_size):
                         class_marks[i_idx + k, j_idx + l] = max_idx
                draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx + win_size),
                                  (i_idx, j_idx + win_size)], outline=t_color[max_idx], fill=t_color[max_idx])
            batch_count = 0
            batch_coords = []
            im = Image.open(i)
            batch_pack[batch_count] = np.array(im) * 1. / 255.
            batch_coords.append(img_coords[idx_counter])
            batch_count += 1
            idx_counter += 1
            im.close()

    img_new.save(result_fileName + str(idx_model) + ".BMP")
    img_new.close()
    return class_marks

class_marks = classification_by_batch(model, img_list_0, img_coords_0)
#class_marks = classification_by_batch(model)



In [ ]:
if probs_gen == True:

    def writing_data_in_qtree(width_re,height_re, resource, result):
        temple_file = Image.open(img_file_name)
        im2 = np.zeros((1, win_size, win_size, 3), dtype='float32')
        class_prbs = np.zeros((width_re, height_re, class_amount), dtype='float32')

        for i in range(0, width_re, step_len - shift_draw):
            for j in range(0, height_re, step_len - shift_draw):
                if (i + win_size) < width_re and (j + win_size) < height_re:
                    im2[0] = np.array(temple_file.crop((i, j, i + win_size, j + win_size))) * 1. / 255.
                    z = model.predict(im2, verbose=0)
                    for k in range(win_size):
                        for l in range(win_size):
                            for p in range(class_amount):
                                class_prbs[i + k, j + l, p] = z[0, p]
                else:
                    i_1, j_1 = i, j
                    if i + win_size >= width_re:
                        i_1 = width_re - win_size
                    if j + win_size >= height_re:
                        j_1 = height_re - win_size
                    im2[0] = np.array(temple_file.crop((i_1, j_1, i_1 + win_size, j_1 + win_size))) * 1. / 255.
                    z = model.predict(im2, verbose=0)
                    for k in range(win_size):
                        for l in range(win_size):
                            for p in range(class_amount):
                                class_prbs[i_1 + k, j_1 + l, p] = z[0, p]
        temple_file.close()
        with open(result, 'w') as file:
            file.write(str(class_amount))
            file.write("\n")
            file.write(str(height_re))
            file.write("\n")
            for i in range(height_re):
                for j in range(width_re):
                    for k in range(class_amount):
                        file.write(str(class_prbs[j, height_re - i - 1, k]))
                        file.write(" ")
                    file.write(" ")
                file.write("\n")
            file.close()
        temple_file.close()

    def writing_data_in_qtree_by_batch(width_re, height_re, resource, result, img_list, img_coords):
        temple_file = Image.open(resource)
        im2 = np.zeros((1, win_size, win_size, 3), dtype='float32')
        class_prbs = np.zeros((width_re, height_re, class_amount), dtype='float32')

        batch_size_t = 64
        batch_count = 0
        batch_pack = np.zeros((batch_size_t, win_size, win_size, 3), dtype='float32')
        batch_coords = []
        idx_counter = 0

        for img_ptr in img_list:

            if (batch_count < batch_size_t):
                im = Image.open(img_ptr)
                batch_pack[batch_count] = np.array(im) * 1. / 255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
            else:
                pred_res = model.predict(batch_pack, batch_size=batch_size_t, verbose=0)
                for j_ptr in range(pred_res.shape[0]):
                    i = batch_coords[j_ptr][0]
                    j = batch_coords[j_ptr][1]
                    if (i + win_size) < width_re and (j + win_size) < height_re:
                        for k in range(win_size):
                            for l in range(win_size):
                                for p in range(class_amount):
                                    class_prbs[i + k, j + l, p] = pred_res[j_ptr, p]
                batch_count = 0
                batch_coords = []
                im = Image.open(img_ptr)
                batch_pack[batch_count] = np.array(im) * 1. / 255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()

        temple_file.close()
        if (batch_count < batch_size_t) and batch_count > 0:
            pred_res = model.predict(batch_pack, batch_size=batch_count, verbose=0)
            for j_ptr in range(pred_res.shape[0]):
                i = batch_coords[j_ptr][0]
                j = batch_coords[j_ptr][1]
                if (i + win_size) < width_re and (j + win_size) < height_re:
                    for k in range(win_size):
                        for l in range(win_size):
                            for p in range(class_amount):
                                class_prbs[i + k, j + l, p] = pred_res[j_ptr, p]
        with open(result, 'w') as file:
            file.write(str(class_amount))
            file.write("\n")
            file.write(str(height_re))
            file.write("\n")
            for i_ptr in range(height_re):
                for j_ptr in range(width_re):
                    for k_ptr in range(class_amount):
                        file.write(str(class_prbs[j_ptr, height_re - i_ptr - 1, k_ptr]))
                        file.write(" ")
                    file.write(" ")
                file.write("\n")
            file.close()
        temple_file.close()


    writing_data_in_qtree_by_batch(width_re, height_re, img_file_name, res_filename_0, img_list_0, img_coords_0)
    writing_data_in_qtree_by_batch(width_re // 2, height_re // 2, glob_dir_name + "accumulated.BMP", res_filename_1, img_list_1, img_coords_1)
    writing_data_in_qtree_by_batch(width_re // 4, height_re // 4, glob_dir_name + "accumulated_0.BMP", res_filename_2, img_list_2, img_coords_2)
    writing_data_in_qtree_by_batch(width_re // 8, height_re // 8, glob_dir_name + "accumulated_1.BMP", res_filename_3, img_list_3, img_coords_3)
    

In [ ]:

with open(marks_filename, 'w') as file:
    for i in range(height_re):
        for j in range(width_re):
            file.write(str(class_marks[j, height_re-i] + 1))
            file.write(" ") 
        file.write("\n")
    file.close()    


# In[119]:


gridsize = (1, 2)
fig = plt.figure(figsize=(18, 16))
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
data = np.loadtxt(marks_filename)
pc = ax1.contourf(data)
plt.colorbar(pc, ax=ax1, format='$%d')
plt.show()